In [1]:
factor = 1.0
exactDate_mode = 'xor'

# Feature Matrix Generation

In chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb), Swissbib's goldstandard data has been processed to form records of pairs of duplicate and pairs of unique records. These records are the starting point for the final feature matrix generation and that is the reason, why the DataFrame was called feature base. As described in [[JudACaps](./A_References.ipynb#judacaps)], the next step will be an attribute-wise comparison of each attribute pair of each record in the original feature base. This comparison will generate similarity values for each attribute pair. It will halve the number of attributes replacing each attribute pair with one value expressing their degree of similarity. The goal of this chapter is a DataFrame with the full and final feature attributes. The values of these feature attributes will be used for training and performance testing of the machine learning models in the chapters to follow.

This chapter introduces similarity metrics for string comparisons. The metrics to be used for calculating its similarity will be decided for each attribute pair of the DataFrame built in the previous chapters.

## Table of Contents

- [Data Takeover](#Data-Takeover)
- [Object Distance and Similarity](#Object-Distance-and-Similarity)
    - [Mathematical Definitions](#Mathematical-Definitions)
    - [Library TextDistance](#Library-TextDistance)
- [Similarity Metrics on Attribute Level](#Similarity-Metrics-on-Attribute-Level)
    - [Table of Contents of Attribute Similarities](#Table-of-Contents-of-Attribute-Similarities)
- [DataFrame with Attributes and Similarity Features](#DataFrame-with-Attributes-and-Similarity-Features)
- [Summary](#Summary)
    - [Full Feature Matrix with Target Vector Handover](#Full-Feature-Matrix-with-Target-Vector-Handover)

## Data Takeover

Swissbib's raw data of the goldstandard has been processed in chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb). As the first step of this chapter, this data is loaded for further processing to the feature matrix and target vector for the subsequent machine learning model chapters.

In [2]:
import os
import pandas as pd
import pickle as pk
import bz2
import _pickle as cPickle

path_goldstandard = './daten_goldstandard'

# Restore metadata so far
with open(os.path.join(path_goldstandard, 'columns_metadata.pkl'), 'rb') as handle:
    columns_metadata_dict = pk.load(handle)

# Restore DataFrame with features from compressed pickle file
with bz2.BZ2File((os.path.join(
    path_goldstandard, 'feature_base_df.pkl')), 'rb') as file:
    df_feature_base = cPickle.load(file)

# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(df_feature_base.columns)

df_feature_base.head()

,035liste_x,035liste_y,century_x,century_y,coordinate_E_x,coordinate_E_y,coordinate_N_x,coordinate_N_y,coordinate_x,coordinate_y,corporate_110_x,corporate_110_y,corporate_710_x,corporate_710_y,corporate_full_x,corporate_full_y,decade_x,decade_y,docid_x,docid_y,doi_x,doi_y,duplicates,edition_x,edition_y,exactDate_x,exactDate_y,format_postfix_x,format_postfix_y,format_prefix_x,format_prefix_y,isbn_x,isbn_y,ismn_x,ismn_y,masters_docid,musicid_x,musicid_y,pages_x,pages_y,part_x,part_y,person_100_x,person_100_y,person_245c_x,person_245c_y,person_700_x,person_700_y,pubinit_x,pubinit_y,pubword_x,pubword_y,pubyear_x,pubyear_y,scale_x,scale_y,ttlfull_245_x,ttlfull_245_y,ttlfull_246_x,ttlfull_246_y,ttlpart_x,ttlpart_y,volumes_x,volumes_y
0,[(RERO)R006024329],[(RETROS)oai:agora.ch:apk-002:2005:284::157],uuuu,2005,,,,,[],[],,,,,,,uuuu,2005,247704504,318419599,,10.5169/seals-377277,0,,,uuuuuuuu,2005uuuu,020000,010053,bk,bk,[],[],,,50438953X,,,[22 p.],[],,284(2005),mozartwolfgang amadeus,bührerwalter,wolfgang amadeus mozart ; emanuel schikaneder ...,[walter bührer],"mendelhermann, schikanederemanuel",,s. mode's verlag (gustav mode),,[S. Mode's Verlag (Gustav Mode)],[],uuuuuuuu,2005,,,"die zauberflöte, oper in zwei akten",blick in die welt,,,"{'245': ['Die Zauberflöte', 'Oper in zwei Akte...",{'245': ['Blick in die Welt']},22 p.,
1,"[(SNL)vtls001092110, (Sz)001092110]",[(RERO)R008551762],1996,2016,,,,,[],[],,,,,,,1996,2016,065307801,403228271,,,0,,,19969999,2016uuuu,030300,020053,cr,bk,[],[978-2-226-31734-6],,,504389955,,,[ v.],[1 livre électronique (e-book)],,,,moriartyliane,,liane moriarty ; traduit de l'anglais (austral...,,taupeaubéatrice,universitätsverlag,albin michel,[Universitätsverlag],[Albin Michel],19969999,2016,,,"bildungsforschung und bildungspraxis, educatio...","petits secrets, grands mensonges","educazione e ricerca., education et recherche....",,{'245': ['Bildungsforschung und Bildungspraxis...,"{'245': ['Petits secrets, grands mensonges']}",v.,1 livre électronique (e-book)
2,[(RERO)1553399],"[(OCoLC)1001961995, (NEBIS)011047950]",1984,2017,,,,,[],[],,,interkantonale lehrmittelzentrale (luzern),,interkantonale lehrmittelzentrale (luzern),,1984,2017,212984225,500162255,,,0,,7,19849999,2017uuuu,020000,020000,bk,bk,[],[978-3-13-240799-2],,,504389599,,,[ v.],[334 Seiten],,,,trappehans-joachim,sigrid kessler... [et al.] ; [éd.:] interkanto...,"hans-joachim trappe, hans-peter schuster",kesslersigrid,schusterhans-peter,staatlicher lehrmittelverlag,,[Staatlicher Lehrmittelverlag],[],19849999,2017,,,"bonne chance!, cours de langue française, troi...",ekg-kurs für isabel,,,"{'245': ['Bonne chance!', 'cours de langue fra...",{'245': ['EKG-Kurs für Isabel']},v.,334 seiten
3,"[(VAUD)991004649259702852, (RNV)008339403-41bc...","[(OCoLC)945563378, (NEBIS)010612600]",2016,2016,,,,,[],[],,,,,,,2016,2016,41487336X,358479975,,,0,,,2016uuuu,2016uuuu,020000,020000,bk,bk,[978-2-07-046833-1],[978-0-7294-1157-8],,,504388916,,,[153 p.],[332 Seiten],"3870., 3870",13c,voltaire,voltaire,voltaire ; éd. établie et annotée par jacques ...,voltaire ; [sous la direction de diego venturino],"van den heuveljacques, sollersphilippe",venturinodiego,gallimard,,[Gallimard],[],2016,2016,,,"traité sur la tolérance, à l'occasion de la mo...","siècle de louis xiv (v), chapitres 25-30",,,"{'245': ['Traité sur la tolérance', 'à l'occas...","{'245': ['Siècle de Louis XIV (V)', 'chapitres...",153 p.,332 seiten
4,"[(OCoLC)806968650, (SGBN)000443345]","[(OCoLC)612031656, (IDSBB)001972916]",2000,1981,,,,,[],[],,,the metropolitan chorus und orchestra,,the metropolitan chorus und orchestra,,2000,1981,048543322,097287970,,,0,,3,2000uuuu,1981uuuu,010300,020000,vm,bk,[],[3-442-33001-7],,,504388967,073,,[1 DVD (ca. 169 Min.)],[252 S.],,"33001, 33001",mozartwolfgang amadeus,mozartwolfgang amadeus,w. a. mozart ; dir.: james livine ; the metrop...,wolgang amadeus mozart ; dieser opernführer wu...,"levinejames, hockneydavid","schikanederemanuel, pahlenkurt",deutsche grammophon,,[Deut

Now, the feature base can be reduced to the columns used for processing.

In [3]:
# The DataFrame of pairs with target information
df_feature_base = df_feature_base[columns_metadata_dict['columns_to_use']]

df_feature_base.sample(n=5)

,duplicates,coordinate_E_x,coordinate_E_y,coordinate_N_x,coordinate_N_y,corporate_full_x,corporate_full_y,doi_x,doi_y,edition_x,edition_y,exactDate_x,exactDate_y,format_prefix_x,format_prefix_y,format_postfix_x,format_postfix_y,isbn_x,isbn_y,ismn_x,ismn_y,musicid_x,musicid_y,part_x,part_y,person_100_x,person_100_y,person_700_x,person_700_y,person_245c_x,person_245c_y,pubinit_x,pubinit_y,scale_x,scale_y,ttlfull_245_x,ttlfull_245_y,ttlfull_246_x,ttlfull_246_y,volumes_x,volumes_y
56487,0,e0060811,,n0461242,,,,,,,,17001799,18871890,mp,bk,010700,020000,[],[],,,,,,"2620, bd. 5, 2620, 5",,mozartwolfgang amadeus,,"schikanederemanuel, wittmanncarl friedrich",,von w.a. mozart ; dichtung nach ludwig gieseck...,,p. reclam jun.,,,"genéve, genff","die zauberflöte, oper in zwei aufzügen : volls...",,,1 ansicht,92 s.
36302,0,,,,,oper (köln),,,,,,19911794,1763uuuu,bk,bk,020000,020000,[],[],,,,,,,mozartwolfgang amadeus,voltaire,schikanederemanuel,,[von emmanuel schikaneder] ; [die musik ist vo...,[voltaire],stadt köln,,,,"die zauberflöte, textbuch : köln 1794",traité sur la tolérance,,,72 s.,211 s.
92155,0,,,,,,,,,,,1987uuuu,2004uuuu,bk,bk,020000,020347,[],[],,,,,,,,biglermatthias,binderheidy,,heidy binder... [et al.] ; [éd.:] interkantona...,matthias bigler,staatlicher lehrmittelverlag,,,,"bonne chance!, cours de langue française, prem...",hochauflösende spurenstoffmessungen an polaren...,,,148 p.,1 cd-rom
87172,0,,,,,,,,,,,uuuuuuuu,1990uuuu,cr,bk,030300,020000,[],[0-19-282756-1],,,,,,,,austenjane,,,,jane austen,idealibri,oxford university press,,,arts,emma,,,,445 p.
63204,0,,,,,,,,,,,2005uuuu,1992uuuu,bk,bk,020000,020053,[0-521-82437-0],[978-3-598-31500-8 (print)],,,,,,"band 20, band 20",austenjane,mortzfeldpeter,croninrichard,raabepaul,jane austen ; ed. by richard cronin and doroth...,"mortzfeld, peter; raabe, paul",cambridge univ. press,de gruyter saur,,,emma,katalog der graphischen porträts in der herzog...,,,"lxxvii, 600 p.","1 online resource (iv, 418 p.)"


In [4]:
print('Number of rows labelled as duplicates {:,d}'.format(len(df_feature_base[
    df_feature_base.duplicates==1])))
print('Number of rows labelled as uniques {:,d}'.format(len(df_feature_base[
    df_feature_base.duplicates==0])))
print('Total number of rows in DataFrame {:,d}'.format(df_feature_base.shape[0],
      'number of columns', df_feature_base.shape[1]))

Number of rows labelled as duplicates 1,473
Number of rows labelled as uniques 103,182
Total number of rows in DataFrame 104,655


In [5]:
print('Part of duplicates (1) and uniques (0) in units of [%]')
print(round(100*df_feature_base.duplicates.value_counts(normalize=True), 2))

Part of duplicates (1) and uniques (0) in units of [%]
0    98.59
1     1.41
Name: duplicates, dtype: float64


DataFrame feature base is the starting point used for the further processing in this chapter.

## Object Distance and Similarity

A mathematical idea of distance and similarity is needed for understanding object pair comparison. This section starts with a motivation for calculating similarities and afterwards gives a very basic definition of the two central terms, distance and similarity. The text of this section is a summary of [[Chri2012](./A_References.ipynb#chri2012)].

### Mathematical Definitions

The attributes to be used for pair comparison may contain values of poor quality. The quality originates in the way the data has been entered at the very source. Manual data entry may suffer from mistyping, automatically scanned data may suffer from insufficiencies of the scanned base material or the recognition algorithm in the optical character recognition (OCR) processing. The basic step of a deduplication process is to identify the probability of two strings of a pair to be a pair of duplicates. This is done by calculating a similarity value between the two strings compared, rather than using an exact comparison function. Based on this common similarity value for an attribute pair, their being duplicates can be decided.

The term similarity is strongly coupled to the term of distance of two values of an attribute. Mathematically, a distance can be explained with the help of a distance function. A _distance function_ or _distance metric_ $dist(o_i, o_j)$ between two points or data objects $o_i$ and $o_j$ must fulfill four requirements.

1. $dist(o_i, o_i)=0$, the distance from an object to itself is zero.
- $dist(o_i, o_j)\ge 0$, the distance between two objects is a non-negative number.
- $dist(o_i, o_j)=dist(o_j, o_i)$, the distance between two objects is symmetric.
- $dist(o_i, o_j)\le dist(o_i, o_k)+dist(o_k, o_j)$, the triangular inequality must hold. It states that the direct distance beween two objects is never larger than the combined distance when going through a third object.

A distance value expresses the dissimilarity $d$ of two objects [[HanK2012](./A_References.ipynb#hank2012)] and can therefore be converted into a similarity value $s$, calculating $s = \frac{1}{d}$, assuming $d\gt 0$. Alternatively, assuming the distance value is normalised $0\le d\le 1$, the similarity value can be calculated to $s = 1-d$. A _similarity function_ $sim(a_i, aj)$ between two attributes which can be strings, numbers, dates, geographic locations, text, XML documents, etc. fulfills the general requirements.

1. $sim(a_i, a_i)=1$, the result of comparing a value with itself is an exact similarity.
- $sim(a_i, a_j)=0$, the similarity of values that are completely different from each other is 0. What accounts for 'complete different' depends upon the type of data that are compared.
- $0\lt sim(a_i, a_j)\lt 1$, an approximate similarity between exact similarity and total dissimilarity is calculated if two attribute values are somewhat similar to each other. What accounts for 'somewhat different' depends upon the type of data that are compared.

The dissimilarity between two objects $o_i$ and $o_j$ can be computed based on the ratio of mismatches,
$$
d(o_i, o_j) = \frac{p-m}{p},
$$
where $m$ is the number of matching attributes and $p$ is the total number of attributes describing the objects [[HanK2012](./A_References.ipynb#hank2012)]. Thus the similarity between two objects can be computed as
$$
sim(o_i, o_j) = 1 - d(o_i, o_j) = \frac{m}{p}.
$$

For data deduplication, a comparison function needs to be tailored to the type of underlying data. Although there is a correspondence between a similarity function and the mathematical concept of a distance function, not all known and implemented similarity comparison functions used for string pair comparison fulfill the requirements of a distance function. Some similarity functions are not symmetric, others do not fulfill the triangular inequality. Decision taking on the best similarity function for a string pair, will be based on the effect, a similarity function has for the purpose needed. In the case of this capstone project, this purpose is its capability to contribute to the prediction whether a pair of records is a pair duplicates or a pair of uniques.

### Library TextDistance

An internet research on string distance calculation with Python has revealed libraries [[StSi](./A_References.ipynb#stsi)], [[TeDi](./A_References.ipynb#tedi)] and seperate code snippets for distinct algorithms. After trying the referenced libraries and a downloaded code snippet for a Smith Waterman similarity [[SmWa](./A_References.ipynb#smwa)], the text distance library [[TeDi](./A_References.ipynb#tedi)] has been decided to be the best decision for this capstone project. The decision is based on the github statistics of stars and the date of the latest pull requests, indicating its popularity and maintenance activity of the library. A look at the API of the library, reveals the Python library to be a complete implementation (compared to suggestions of similarity metrics in [[Chri2012](./A_References.ipynb#chri2012)]) and easy to use.

In [6]:
# Install textdistance Python library - if not done, yet.
! pip install textdistance

For using the library, see documentation in [[TeDi](./A_References.ipynb#tedi)]. For the purposes of this chapter, function $\texttt{.normalized}\_\texttt{similarity}()$ of an instantiated textdistance object will be used.

In [7]:
import textdistance as tedi

With the code line above, the library is imported for application in this chapter. In appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb) the effects of the similarity metrics of the library are compared for better understanding of their specific behaviour. This comparison for each attribute is the basis of deciding the best similarity metric available for an attribute pair.

## Similarity Metrics on Attribute Level

This section implements the decision for calculating the similarity metric for each attribute of the raw data based on appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb). The implementation is applied on a pair of attributes of two records, resulting in a new attribute, the similarity value, of the final feature matrix. A general function $\texttt{.build}\_\texttt{delta}\_\texttt{feature}()$ is provided by the code file [data_preparation_funcs.py](./data_preparation_funcs.py) for transforming two attributes into their feature attribute holding their similarity value.

In [8]:
import data_preparation_funcs as dpf

The two dictionaries of the following code cell will be filled by function $\texttt{.build}\_\texttt{delta}\_\texttt{feature}()$.

In [9]:
columns_metadata_dict['similarity_metrics'] = {}
columns_metadata_dict['features'] = []

### Table of Contents of Attribute Similarities

- [coordinate](#coordinate)
- [corporate](#corporate)
- [doi](#doi)
- [edition](#edition)
- [exactDate](#exactDate)
- [format](#format)
- [isbn](#isbn)
- [ismn](#ismn)
- [musicid](#musicid)
- [part](#part)
- [person](#person)
- [pubinit](#pubinit)
- [scale](#scale)
- [ttlfull](#ttlfull)
- [volumes](#volumes)

### coordinate

As discussed in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{coordinate}$ holds coordinates of maps. To decide whether a map covers the same geographical range, a metric will be chosen that compares the coordinate number digits from left to right. The more digits are found to be equal, the higher the similarity value is calculated. The comparison stops with the first digit pair that differs. This algorithm is satisfied by the LCS (Longest Common Substring comparison) algorithm and generates the wanted result, see appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb).

In [10]:
attribute = 'coordinate'

columns_metadata_dict['similarity_metrics'][attribute+'_E'] = tedi.LCSStr()
columns_metadata_dict['similarity_metrics'][attribute+'_N'] = tedi.LCSStr()

ne_values = ['_E', '_N']

for ne in ne_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+ne,
        columns_metadata_dict['similarity_metrics'][attribute+ne],
        columns_metadata_dict)

The length of attribute $\texttt{coordinate}$ is exactly eight digits. The distinct similarity values that may occur form a discrete set of values with a distance of $\frac{1}{8}$ between adjacent values.

In [11]:
uniques, uniques_len = {}, {}

for ne in ne_values :
    uniques[attribute+ne], uniques_len[attribute+ne] = dpf.determine_similarity_values(
        df_feature_base, attribute+ne)

coordinate_E values range [0.    0.375 0.5   0.625 0.875 1.   ]
coordinate_N values range [0.    0.375 0.5   0.75  0.875 1.   ]


Looking at some samples of the feature matrix reveals a good match to the expectations.

In [12]:
position = 3

for ne in ne_values :
    dpf.show_samples_interval(
        df_feature_base, attribute+ne,
        uniques[attribute+ne][uniques_len[attribute+ne]-position],
        uniques[attribute+ne][uniques_len[attribute+ne]-position+1]
    )

,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y
11796,0,0.875,e0080851,e0080855
64041,0,0.875,e0080851,e0080855
104051,1,0.875,e0080855,e0080851
104047,1,0.875,e0080851,e0080855
104045,1,0.875,e0080851,e0080855


0.625 <= coordinate_E_delta <= 0.875


,duplicates,coordinate_N_delta,coordinate_N_x,coordinate_N_y
29979,0,0.750,n0460826,n0460833
64041,0,0.875,n0460821,n0460826
96644,0,0.750,n0460826,n0460833
100879,0,0.750,n0460826,n0460833
7347,0,0.750,n0460826,n0460833


0.75 <= coordinate_N_delta <= 0.875


The samples above show the wanted similarity behaviour for value ranges greater than 0. The metric has the weakness, though, that empty coordinate values, e.g. for bibliographic units other than maps, have each been calculated to a similarity of 0. Some samples for duplicates in the training data are shown below.

In [13]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1],
    attribute+'_E', uniques[attribute+'_E'][0], uniques[attribute+'_E'][1], 10)

,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y
103725,1,0.0,,
103941,1,0.0,,
103637,1,0.0,,
103775,1,0.0,,
104348,1,0.0,,
103520,1,0.0,,
104442,1,0.0,,
103650,1,0.0,,
103781,1,0.0,,
103655,1,0.0,,


0.0 <= coordinate_E_delta <= 0.375


This downside shall be avoided by marking pairs of missing coordinate values on both sides with a special negative value, which will point out to the models to be trained, the special case of missing information in a row. The implementation of this logic is done in function $\texttt{.mark}\_\texttt{missing}()$. The absolute value of this negative number is conrolled by a factor which is passed to the function as a parameter. The function handles explicitly two cases. The first one is missing information in both attributes (resulting in $-1*\texttt{factor}$) and the second one is missing information in only one of the two attributes (resulting in $-0.5*\texttt{factor}$).

In [14]:
for ne in ne_values :
    df_feature_base = dpf.mark_missing(df_feature_base, attribute+ne, factor)

### corporate

Attribute $\texttt{corporate}$ is a collection of corporate names. The Monge-Elkan metric compares string tokens pairwise [[Chri2012](./A_References.ipynb#chri2012)] while the LCS metric searches for the longest common substring. Assessing the differences of these two metrics with the help of their values distribution in chapter [Features Discussion and Dummy Classifier Baseline](./5_FeatureDiscussionDummyBaseline.ipynb), reveals a better distribution behaviour for LCS. Therefore, the LCS metric will be chosen for this attribute.

In [15]:
attribute = 'corporate_full'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.LCSStr()
#tedi.StrCmp95()
#tedi.MongeElkan()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [16]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

corporate_full values range [0.         0.01333333 0.01428571 0.01639344 0.01666667 0.01694915
 0.01818182 0.02       0.02173913 0.02272727 0.02380952 0.02531646
 0.02542373 0.02666667 0.02702703 0.02727273 0.02857143 0.03
 0.03030303 0.03125    0.03278689 0.03333333 0.03389831 0.03448276
 0.03571429 0.03636364 0.03773585 0.03797468 0.03846154 0.03921569
 0.04       0.04081633 0.04166667 0.04237288 0.04255319 0.04285714
 0.04347826 0.04444444 0.04545455 0.046875   0.04761905 0.04878049
 0.04918033 0.05       0.05263158 0.05333333 0.05405405 0.05454545
 0.05555556 0.05660377 0.05714286 0.05882353 0.05932203 0.06
 0.06060606 0.06122449 0.0625     0.06329114 0.06363636 0.06382979
 0.06451613 0.06521739 0.06557377 0.06666667 0.06779661 0.06818182
 0.06896552 0.07       0.07142857 0.07272727 0.07317073 0.075
 0.0754717  0.07627119 0.07692308 0.078125   0.07843137 0.07894737
 0.08       0.08108108 0.08163265 0.08181818 0.08333333 0.08474576
 0.08510638 0.08571429 0.08695652 0.08888889 0.0909

Its $110$ part is sparsely filled and even its $710$ part comes along with a little more than $10\%$ of filling, only. The LCS metric generates a similarity of 1 for the cases where both strings of a pair are empty. Missing values on both sides may be an indicator for a pair of duplicates but due to the sparsely available information, it is a weak indicator. Therefore, the pairs with missing data on both sides of the pair, will be marked with the negative value.

In [17]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

Some sample cases are shown below for both $\texttt{corporate}$ features.

In [18]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1],
    attribute, 0.0, 1.0, 20
)

,duplicates,corporate_full_delta,corporate_full_x,corporate_full_y
104036,1,1.00,eidgenössische landestopographie,eidgenössische landestopographie
103617,1,1.00,wiener philharmoniker,wiener philharmoniker
104564,1,1.00,schweizerische normen-vereinigung,schweizerische normen-vereinigung
103611,1,1.00,wiener philharmoniker,wiener philharmoniker
104595,1,1.00,schweizerische normen-vereinigung,schweizerische normen-vereinigung
104491,1,1.00,schweizeidgenössisches topographisches bureau,schweizeidgenössisches topographisches bureau
103351,1,0.49,schweizerische gesellschaft für bildungsforsch...,schweizerische gesellschaft für bildungsforschung
103390,1,1.00,schweizerische gesellschaft für bildungsforschung,schweizerische gesellschaft für bildungsforschung
103940,1,1.00,société suisse pour la recherche en éducation,société suisse pour la recherche en éducation
104607,1,1.00,schweizerische normen-vereinigung,schweizerische normen-vereinigung


0.0 <= corporate_full_delta <= 1.0


In [19]:
position = uniques_len[attribute]//2 # Let's have a look in the middle range of the similarities.

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 20)

,duplicates,corporate_full_delta,corporate_full_x,corporate_full_y
30318,0,0.093333,opernhaus orchester zürich,"metropolitan opera (new york)chorus, metropoli..."
65374,0,0.088889,société suisse pour la recherche en éducation,eidgenössische landestopographie
98573,0,0.090909,theater st. gallen,philadelphia orchestra
19781,0,0.090909,arts florissants,schweizerische normen-vereinigung
68169,0,0.090909,schweizerische normen-vereinigung,arts florissants
8229,0,0.090909,philharmonia orchestra (london),schweizerische normen-vereinigung
52464,0,0.090909,les arts florissants,schweizerische normen-vereinigung
12276,0,0.088889,société suisse pour la recherche en éducation,eidgenössische landestopographie
61253,0,0.090909,schweizerische normen-vereinigung,the metropolitan opera
54683,0,0.090909,interkantonale lehrmittelzentrale,eidgenössische landestopographie


0.0888888888888889 <= corporate_full_delta <= 0.09333333333333338


### doi

Swissbib uses an explicit $\texttt{doi}$ attribute for its deduplication implementation. In chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb), the real doi identifier has been isolated with the help of a preprocessing function $\texttt{.reduce}\_\texttt{to}\_\texttt{doi}\_\texttt{element}()$, see [Data Analysis](./1_DataAnalysis.ipynb). Attribute $\texttt{doi}$ contains a single string value. The Identity metric will be used for comparing the string values of a pair in a row, calculating a similarity value of 1.0 or 0.0 for each pair. If one list is empty a value of 0 is returned.

In [20]:
attribute = 'doi'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Identity()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

df_feature_base['doi_delta'].unique()

array([0., 1.])

Some sample cases are shown below for each category of $\texttt{doi}\_\texttt{delta}$.

In [21]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

for doi_delta_value in df_feature_base['doi_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['doi_delta']==doi_delta_value])
    )

    dpf.show_samples_distinct(df_feature_base, 'doi', doi_delta_value, number_of_max_samples)
    print(f'doi_delta = {doi_delta_value}')

doi values range [0. 1.]


,duplicates,doi_delta,doi_x,doi_y
11764,0,0.0,10.5451/unibas-006499413,
95099,0,0.0,,10.5169/seals-376510
46578,0,0.0,,10.5169/seals-376689
90250,0,0.0,10.1007/978-3-642-41698-9,
38647,0,0.0,,10.5169/seals-376396
51358,0,0.0,,10.1007/978-3-642-41698-9
46883,0,0.0,,10.3931/e-rara-50904
91673,0,0.0,,10.5169/seals-376810
97068,0,0.0,,10.5169/seals-376539
34492,0,0.0,10.5451/unibas-006499413,


doi_delta = 0.0


,duplicates,doi_delta,doi_x,doi_y
85676,0,1.0,,
36368,0,1.0,,
49783,0,1.0,,
17707,0,1.0,,
66265,0,1.0,,
74989,0,1.0,,
68691,0,1.0,,
65154,0,1.0,,
19769,0,1.0,,
92767,0,1.0,,


doi_delta = 1.0


In [22]:
# Let's have a look at some non-empty doi elements
df_doi_with_element = df_feature_base[df_feature_base.doi_x.apply(lambda x : len(x) > 0)]

for doi_delta_value in df_feature_base['doi_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['doi_delta']==doi_delta_value])
    )

    dpf.show_samples_distinct(df_doi_with_element, 'doi', doi_delta_value, number_of_max_samples)
    print(f'doi_delta = {doi_delta_value}')

,duplicates,doi_delta,doi_x,doi_y
11122,0,0.0,10.1007/978-3-642-41698-9,
96385,0,0.0,10.1093/ndt/gft319,
99323,0,0.0,10.1007/978-3-642-41698-9,
84508,0,0.0,10.1093/ndt/gft319,
51159,0,0.0,10.1093/cid/ciu795,
51021,0,0.0,10.1093/ndt/gft319,
89753,0,0.0,10.1093/ndt/gft319,
50361,0,0.0,10.1093/cid/cir669,
23086,0,0.0,10.1055/b-005-143650,
71913,0,0.0,10.1093/cid/ciu795,


doi_delta = 0.0


,duplicates,doi_delta,doi_x,doi_y
104445,1,1.0,10.1093/ndt/gft319,10.1093/ndt/gft319
104448,1,1.0,10.1093/cid/ciu795,10.1093/cid/ciu795
104651,1,1.0,10.1055/b-005-143650,10.1055/b-005-143650
104449,1,1.0,10.1093/cid/ciu795,10.1093/cid/ciu795
104442,1,1.0,10.1007/978-3-642-41698-9,10.1007/978-3-642-41698-9
104367,1,1.0,10.5167/uzh-53042,10.5167/uzh-53042
104441,1,1.0,10.1007/978-3-642-41698-9,10.1007/978-3-642-41698-9
104433,1,1.0,10.1055/b-002-26639,10.1055/b-002-26639
104446,1,1.0,10.1093/ndt/gft319,10.1093/ndt/gft319
104375,1,1.0,10.1093/cid/cir669,10.1093/cid/cir669


doi_delta = 1.0


As can be seen above, a value of 1.0 is returned if both strings of a pair are empty. As the attribute filling of $\texttt{doi}$ is sparse, see chapter [Data Analysis](./1_DataAnalysis.ipynb), the $\texttt{doi}\_\texttt{delta}$ indicates strongly a pair of duplicates for most rows. To avoid such misleading identity indication, function $\texttt{.mark}\_\texttt{missing}()$ will be applied to the attribute.

In [23]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### edition

In its original form in Swissbib's raw data, the edition statement is a string value which may have several words. The modelling on this attribute has been tried with and without stripping letter characters from the string. The final decision for the best processing will be documented in chapter [Overview and Summary](./0_OverviewSummary.ipynb). A Jaccard similarity is tried for this attribute.

In [24]:
attribute = 'edition'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [25]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

import numpy as np

edition_delta_uniques = np.sort(df_feature_base['edition_delta'].unique())
edition_delta_uniques_len = len(edition_delta_uniques)
print('edition values range', edition_delta_uniques[:30])

edition values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.4        0.5        0.6        1.        ]
edition values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.4        0.5        0.6        1.        ]


The comparison results in a wide number of distinct similarity values for the goldstandard data set. Below, some examples are shown.

In [26]:
position = edition_delta_uniques_len

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position],
    edition_delta_uniques[edition_delta_uniques_len-position+2], 10)

position = edition_delta_uniques_len//2

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)

,duplicates,edition_delta,edition_x,edition_y
54741,0,1.0,,
28934,0,1.0,,
456,0,1.0,,
67689,0,1.0,,
33572,0,1.0,,
101192,0,1.0,,
64282,0,1.0,,
23185,0,1.0,,
16156,0,1.0,,
29194,0,1.0,,


0.6 <= edition_delta <= 1.0


,duplicates,edition_delta,edition_x,edition_y
29568,0,0.0,,2
12848,0,0.0,5,
15133,0,0.0,,144
76707,0,0.0,1,
20171,0,0.0,2,
9275,0,0.0,2,3
78755,0,0.0,,2
99157,0,0.0,,8
75236,0,0.0,2,
67677,0,0.0,,2


0.0 <= edition_delta <= 0.1428571428571428


,duplicates,edition_delta,edition_x,edition_y
82835,0,0.25,8,1899
20320,0,0.20,10,1889
22450,0,0.20,10425,2
3665,0,0.20,1791,12
90070,0,0.25,2,1926
47106,0,0.25,1,1899
26816,0,0.25,1791,7
15946,0,0.25,6,1926
50624,0,0.25,6,1926
20363,0,0.25,1,1998


0.19999999999999996 <= edition_delta <= 0.25


Again, for $\texttt{edition}\_\texttt{delta} = 1$, many empty values of the $\texttt{edition}$ attribute can be observed. These will be marked with the special negative value in the data with the goal to distinguish them from the matching attribute pairs.

In [27]:
df_feature_base = dpf.mark_missing(df_feature_base, 'edition', factor)

In [28]:
position = edition_delta_uniques_len

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)

,duplicates,edition_delta,edition_x,edition_y
74029,0,1.0,2,2
70503,0,1.0,3,3
90029,0,1.0,5,5
741,0,1.0,3,3
12579,0,1.0,8,8
104182,1,1.0,3,3
39684,0,1.0,6,6
104165,1,1.0,8,8
104462,1,1.0,3,3
37946,0,1.0,3,3


0.6 <= edition_delta <= 1.0


### exactDate

As discussed in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{exactDate}$ holds a year number stored in the first four digits. Letter 'u' is used as a placeholder for an unknown digit. The attribute may hold some month and day or a second year information in its second four digits, additionally.

The attribute will be kept as a string and will not be transformed to an integer. The feature attribute of the record pair to be compared will be calculated with a modified Hamming algorithm, see appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb). The resulting similarity will be stored in a new attribute $\texttt{exactDate}\_\texttt{delta}$ which will be taken for the model calculation.

As can be seen in chapter [Decision Tree Model](./6_DecisionTreeModel.ipynb), this attribute is important for prediction. Different ways of increasing the weight of the unknown status of a digit have been tried. The different ways can be seen in the implementations below. The algorithm decided for the final simulation will be documented in chapter [Overview and Summary](./0_OverviewSummary.ipynb).

In [29]:
import string

def no_xor (x_side, y_side) :
    number = 0
    for i in range(len(x_side)) :
        if ((x_side[i] in string.ascii_lowercase) | (y_side[i] in string.ascii_lowercase)) & (x_side[i] != y_side[i]) :
            number = number + 1
    return number

print('Example comparison results in a value of', no_xor ('202a0aaa', '1920uuuu'))

Example comparison results in a value of 5


In [30]:
attribute = 'exactDate'

# Replace letter 'u' with letter 'a' for one of the two strings.
#  As an effect, the resulting Hamming similarity for a letter
#  instead of a numerical digit in either string will add with an amount 0.
df_feature_base[attribute+'_x'] = df_feature_base.exactDate_x.str.replace('u', 'a')

# Compute Hamming similarity for century string pair.
columns_metadata_dict['similarity_metrics'][attribute] = tedi.Hamming()

unknown_share = 16

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

if exactDate_mode == 'added_u':
    # Add amount of 1/16 to Hamming similarity for every letter digit.
    #  But only maximum number of letter digits in both strings of a pair.
    df_feature_base[attribute+'_delta'] = df_feature_base[[
        attribute+'_x', attribute+'_y', attribute+'_delta']].apply(
        lambda x : x[attribute+'_delta'] + 
        max(x[attribute+'_x'].count('a'), x[attribute+'_y'].count('u'))/unknown_share, axis=1
    )
elif exactDate_mode == 'xor':
    # Add amount of 1/16 to Hamming similarity for every letter digit.
    #  But only number of position-wise xor-ed letter digits in the two strings of a pair.
    df_feature_base[attribute+'_delta'] = df_feature_base[[
        attribute+'_x', attribute+'_y', attribute+'_delta']].apply(
        lambda x : x[attribute+'_delta'] + 
        no_xor(x[attribute+'_x'], x[attribute+'_y'])/unknown_share, axis=1
    )

In [31]:
df_feature_base[['exactDate_x', 'exactDate_y', 'exactDate_delta']].sample(n=10)

,exactDate_x,exactDate_y,exactDate_delta
88452,1990aaaa,1987uuuu,0.500
76172,1993aaaa,1975uuuu,0.500
9214,2002aaaa,1970uuuu,0.250
6718,2016aaaa,2009uuuu,0.500
52197,1950aaaa,19951995,0.500
87864,aaaaaaaa,2014uuuu,0.500
66667,2004aaaa,1902uuuu,0.375
38253,aaaaaaaa,2008uuuu,0.500
57980,2017aaaa,1900uuuu,0.250
71003,2000aaaa,2007uuuu,0.625


All resulting values of equal strings are equal to 1.

In [32]:
df_feature_base[['exactDate_x', 'exactDate_y', 'exactDate_delta']][
    df_feature_base.exactDate_x == df_feature_base.exactDate_y
].sort_values('exactDate_delta', ascending=False).head()

,exactDate_x,exactDate_y,exactDate_delta
3747,20151475,20151475,1.0
103956,19561791,19561791,1.0
103962,20091990,20091990,1.0
103963,20091990,20091990,1.0
103966,20091990,20091990,1.0


A discrete set of different similarity values can be found in the attribute deltas. Some sample records are shown below.

In [33]:
exactDate_deltas = np.sort(df_feature_base.exactDate_delta.unique())
exactDate_deltas

array([0.    , 0.125 , 0.25  , 0.3125, 0.375 , 0.4375, 0.5   , 0.5625,
       0.625 , 0.6875, 0.75  , 0.875 , 1.    ])

In [34]:
sample_size = 5

for i in exactDate_deltas :
    dpf.show_samples_distinct(df_feature_base, 'exactDate', i, sample_size)
    print(f'exactDate_delta = {i}')

,duplicates,exactDate_delta,exactDate_x,exactDate_y
65111,0,0.0,20150201,19819999
3602,0,0.0,19791999,20172016
53521,0,0.0,19702006,20130627
80892,0,0.0,20151475,19989999
5348,0,0.0,20022000,19959999


exactDate_delta = 0.0


,duplicates,exactDate_delta,exactDate_x,exactDate_y
80028,0,0.125,20159999,19911794
100840,0,0.125,20150201,19959999
79278,0,0.125,20111201,18971989
74004,0,0.125,20151475,19911794
70869,0,0.125,19791999,20151475


exactDate_delta = 0.125


,duplicates,exactDate_delta,exactDate_x,exactDate_y
96911,0,0.25,19702006,19811984
21039,0,0.25,19791999,2008uuuu
73544,0,0.25,2011aaaa,1994uuuu
40037,0,0.25,1932aaaa,2016uuuu
102886,0,0.25,2016aaaa,1995uuuu


...

exactDate_delta = 0.75


,duplicates,exactDate_delta,exactDate_x,exactDate_y
81892,0,0.875,19969999,19959999
100312,0,0.875,19969999,19669999
10563,0,0.875,19819999,19809999
30687,0,0.875,19849999,19809999
104446,1,0.875,201310aa,201310uu


exactDate_delta = 0.875


,duplicates,exactDate_delta,exactDate_x,exactDate_y
103376,1,1.0,19791999,19791999
103341,1,1.0,20022000,20022000
103915,1,1.0,19702006,19702006
104469,1,1.0,20159999,20159999
103963,1,1.0,20091990,20091990


exactDate_delta = 1.0


### format

Due to the discussion in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{format}$ has been split up into two new attributes $\texttt{format}\_\texttt{prefix}$ and $\texttt{format}\_\texttt{postfix}$ which will be compared by a different similarity metrics.

- As the quality of $\texttt{format}\_\texttt{prefix}$ is expected to be high, an identity comparison should be sufficient.
- Due to the observed structure of $\texttt{format}\_\texttt{postfix}$, a q-gram based comparison will be chosen.

In [35]:
attribute = 'format'

columns_metadata_dict['similarity_metrics'][attribute+'_prefix'] = tedi.Identity()
columns_metadata_dict['similarity_metrics'][attribute+'_postfix'] = tedi.Jaccard(qval=2)

pfix_values = ['_prefix', '_postfix']

for pf in pfix_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+pf,
        columns_metadata_dict['similarity_metrics'][attribute+pf],
        columns_metadata_dict)

In [36]:
for i in df_feature_base.format_prefix_delta[
    df_feature_base.format_prefix_x != df_feature_base.format_prefix_y].unique():
    
    dpf.show_samples_distinct(df_feature_base, 'format_prefix', i)
    print(f'format_prefix_delta = {i}')

,duplicates,format_prefix_delta,format_prefix_x,format_prefix_y
51679,0,0.0,bk,vm
66135,0,0.0,vm,mu
25564,0,0.0,bk,vm
82506,0,0.0,cr,bk
81906,0,0.0,vm,bk


format_prefix_delta = 0.0


In [37]:
for i in df_feature_base.format_postfix_delta[
    df_feature_base.format_postfix_x != df_feature_base.format_postfix_y].unique():
    
    dpf.show_samples_distinct(df_feature_base, 'format_postfix', i)
    print(f'format_postfix_delta = {i}')

,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
77911,0,0.111111,010100,020000
5994,0,0.111111,020000,030100
79879,0,0.111111,010100,020053
66779,0,0.111111,010100,020000
45387,0,0.111111,010053,020000


format_postfix_delta = 0.11111111111111116


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
82133,0,0.428571,010200,010053
102768,0,0.428571,020300,020000
54046,0,0.428571,020000,030000
99532,0,0.428571,020000,020053
63117,0,0.428571,020053,020000


format_postfix_delta = 0.4285714285714286


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
34619,0,0.25,020300,030653
11840,0,0.25,020000,020353
102125,0,0.25,010347,010200
81713,0,0.25,020000,020353
37720,0,0.25,010053,010347


format_postfix_delta = 0.25


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
90895,0,0.0,010100,020353
66248,0,0.0,010347,020000
49849,0,0.0,010100,020353
78084,0,0.0,040100,020353
56165,0,0.0,040100,030653


format_postfix_delta = 0.0


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
31565,0,1.0,020000,020000
30095,0,1.0,020000,020000
104571,1,1.0,020053,020053
35573,0,1.0,010100,010100
103522,1,1.0,020000,020000


format_postfix_delta = 1.0


### isbn

Swissbib uses each string element of the $\texttt{isbn}$ list separately for comparing with each string element of its comparison $\texttt{isbn}$ list. If two bibliographic units hold at least one element in common, this is interpreted as a strong indicator for duplicates [[WiCo2001](./A_References.ipynb#wico2001)].

This hard logic is used in a modified way in the context of this capstone project. A special comparison function $\texttt{.build}\_\texttt{delta}\_\texttt{isbn}()$ has been implemented that compares each list element of the left-hand side with each list element of the right-hand side of a pair. According to Swissbib's implementation, the Identity metric is used for string comparison, calculating a similarity value of 1.0 or 0.0 for each list element pair. For normalisation reasons, the sum of similarity values is divided by the number of elements of the smaller list. If both lists are empty a value of 1.0 is returned. If only one list is empty a value of 0.0 is returned.

In [38]:
attribute = 'isbn'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Identity()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

df_feature_base[attribute+'_delta'].unique()

array([1. , 0. , 0.5])

Some sample cases are shown below for each category of $\texttt{isbn_delta}$.

In [39]:
for isbn_delta_value in df_feature_base['isbn_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['isbn_delta']==isbn_delta_value])
    )

    dpf.show_samples_distinct(df_feature_base, 'isbn', isbn_delta_value, number_of_max_samples)
    print(f'isbn_delta = {isbn_delta_value}')

,duplicates,isbn_delta,isbn_x,isbn_y
15616,0,1.0,[],[]
13928,0,1.0,[],[]
67302,0,1.0,[],[]
24980,0,1.0,[],[]
71583,0,1.0,[],[]
40956,0,1.0,[],[]
62889,0,1.0,[],[]
29267,0,1.0,[],[]
98758,0,1.0,[],[]
31420,0,1.0,[],[]


isbn_delta = 1.0


,duplicates,isbn_delta,isbn_x,isbn_y
29234,0,0.0,[978-3-7190-3171-8 (Bd. 1)],"[978-3-598-31499-5 (print), 978-3-11-097005-0]"
55733,0,0.0,[978-2-286-13825-7],"[3-906721-44-2, 3-906721-47-7, 3-906721-45-0, ..."
4702,0,0.0,"[3-495-48796-4, 978-3-495-48796-9]",[]
5223,0,0.0,[3-13-127283-X],[978-3-15-010955-7]
43363,0,0.0,[0252-9955],"[978-3-13-128546-1, 3-13-128546-X, 978-3-13-20..."
97737,0,0.0,[],"[978-3-13-240799-2, 3-13-240799-2]"
78869,0,0.0,[],[978-2-07-042871-7]
86688,0,0.0,"[978-3-448-10147-8, 3-448-10147-8]",[]
93281,0,0.0,[],[3-906721-47-7]
68562,0,0.0,[978-3-643-12370-1],"[978-3-13-128546-1, 3-13-128546-X]"


isbn_delta = 0.0


,duplicates,isbn_delta,isbn_x,isbn_y
104387,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"
104381,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"
104392,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"
104383,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"
104384,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"
104377,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"


isbn_delta = 0.5


For attribute $\texttt{isbn}$, the special marking of missing values is omitted.

### ismn

This attribute will be processed with the identity similarity metric. The reasoning for this decision is the same as for similar attributes above. 

In [40]:
attribute = 'ismn'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Identity()
#tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [41]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

ismn values range [0. 1.]


In [42]:
for ismn_delta_value in df_feature_base[attribute+'_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base[attribute+'_delta']==ismn_delta_value])
    )

    dpf.show_samples_distinct(df_feature_base, 'ismn', ismn_delta_value, number_of_max_samples)
    print(f'ismn_delta = {ismn_delta_value}')

,duplicates,ismn_delta,ismn_x,ismn_y
43512,0,1.0,,
44384,0,1.0,,
12585,0,1.0,,
50116,0,1.0,,
53249,0,1.0,,
47301,0,1.0,,
14976,0,1.0,,
55372,0,1.0,,
39984,0,1.0,,
11502,0,1.0,,


ismn_delta = 1.0


,duplicates,ismn_delta,ismn_x,ismn_y
82950,0,0.0,,m008060205
47660,0,0.0,,m006450510
17343,0,0.0,,m006450510
57964,0,0.0,,m006450510
82254,0,0.0,m006450510,
29797,0,0.0,m006450510,
38258,0,0.0,,9790006201334
20730,0,0.0,"m006546756 (kritischer bericht, leinen)",
103915,1,0.0,,m006546756
91141,0,0.0,m006204687,


ismn_delta = 0.0


As can be seen in the previous chapters, attribute $\texttt{ismn}$ is filled sparsely. A lot of missing values calculate to a value of 1.0 in the chosen similarity metrics. To mark these cases specifically, they will be transformed to a negative value.

In [43]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### musicid

Chapter [Data Analysis](./1_DataAnalysis.ipynb) shows that attribute $\texttt{musicid}$ is an identifyer for a music record. A Jaccard metric has been tested on this attribute, resulting in a distribution of many high similarity values on uniques. Comparing this result with the LCS metric, the latter has been decided.

In [44]:
attribute = 'musicid'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.LCSStr()
#tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [45]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

musicid values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.375      0.4        0.42857143 0.5
 0.66666667 0.75       1.        ]


In [46]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

position = uniques_len[attribute]//2

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)

,duplicates,musicid_delta,musicid_x,musicid_y
104224,1,1.00,4553,4553
103727,1,1.00,4553,4553
103981,1,1.00,502430,502430
8514,0,0.75,063,0630
64919,0,1.00,4553,4553
103458,1,1.00,502023,502023
103465,1,1.00,502023,502023
104145,1,1.00,7794,7794
103762,1,1.00,99036,99036
103963,1,1.00,502430,502430


0.75 <= musicid_delta <= 1.0


,duplicates,musicid_delta,musicid_x,musicid_y
21743,0,0.0,,
98513,0,0.0,,
14240,0,0.0,,
63038,0,0.0,4553,
59783,0,0.0,,
102267,0,0.0,,
57386,0,0.0,,
88503,0,0.0,,542
26156,0,0.0,,
70313,0,0.0,134,


0.0 <= musicid_delta <= 0.1428571428571429


,duplicates,musicid_delta,musicid_x,musicid_y
30649,0,0.4,10425,242
77902,0,0.4,10425,1004
65576,0,0.4,10425,542
32383,0,0.4,10425,426
74536,0,0.4,422,10425
5937,0,0.4,10425,410
72822,0,0.4,10425,542
78082,0,0.4,10425,242
20587,0,0.4,10425,8421
91851,0,0.4,10425,410


0.375 <= musicid_delta <= 0.4


In [47]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1], attribute,
    uniques[attribute][0],
    uniques[attribute][uniques_len[attribute]-1], 20)

,duplicates,musicid_delta,musicid_x,musicid_y
103527,1,0.0,,
103804,1,0.0,,99036
103275,1,0.0,,
103739,1,0.0,,
104440,1,0.0,,
104310,1,0.0,,
103266,1,0.0,,
103287,1,0.0,,
103217,1,0.0,,
103520,1,0.0,,


0.0 <= musicid_delta <= 1.0


The attribute is filled with a degree of below $10\%$. The chosen metric for it results in a similarity value of 1.0 for empty value pairs. This effect can be adjusted with function $\texttt{.mark}\_\texttt{missing}()$ as above. 

In [48]:
df_feature_base = dpf.mark_missing(df_feature_base, 'musicid', factor)

### part

Analogous to attribute $\texttt{edition}$ described above, the string value of this attribute can be stripped to pure number digits. Both ways, with and without letter stripping have been tried for modelling. The final decision for the best processing will be documented in chapter [Overview and Summary](./0_OverviewSummary.ipynb). Three different metrics have been tried for attribute $\texttt{part}$. Finally, metric StringCompare95 will be used.

In [49]:
attribute = 'part'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.StrCmp95()
#tedi.Jaro()
#tedi.Hamming()
#tedi.LCSStr()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [50]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

part values range [0.         0.23917051 0.24458874 ... 0.97142857 0.98666667 1.        ]


In [51]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

position = uniques_len[attribute]//7

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

,duplicates,part_delta,part_x,part_y
34868,0,1.0,,
54596,0,1.0,,
49675,0,1.0,,
5372,0,1.0,,
3861,0,1.0,,
66835,0,1.0,,
31294,0,1.0,,
102511,0,1.0,,
23642,0,1.0,,
42260,0,1.0,,


0.9866666666666667 <= part_delta <= 1.0


,duplicates,part_delta,part_x,part_y
101168,0,0.0,"3870., 3870",
96609,0,0.0,,12
14844,0,0.0,20c,
84363,0,0.0,,12
46159,0,0.0,,[11.4]
84131,0,0.0,,1
65004,0,0.0,,3
13395,0,0.0,2620,
74275,0,0.0,,"band 18, band 18"
54900,0,0.0,ed. 6,


0.0 <= part_delta <= 0.24458874458874458


,duplicates,part_delta,part_x,part_y
24305,0,0.622409,"bl. 285, 285,1963",lief. 2
84835,0,0.623333,"60/3(2015-02-01), 432-437",2191


0.6224089635854342 <= part_delta <= 0.6233333333333334


,duplicates,part_delta,part_x,part_y
69918,0,0.625000,"bd. 5, 5",tp 155
69862,0,0.625000,bd. 8008,band 9
90858,0,0.625000,bd. 63,nr. 2620
49069,0,0.625000,2620,"bl. 23,1896, 23,1896, 23"
59408,0,0.625000,n. 1,"bl. 23,1943, 23,1943, 23"
7880,0,0.625397,nr. 313 (august 2017),z 1
66535,0,0.625000,"[42], 42",[11.4]
43613,0,0.625000,n. 1,"bl. 23,1902, 23,1902, 23"
101058,0,0.625000,f. 285,nr. 2620
80673,0,0.625000,"63, 63",nr. 7633


0.6237373737373737 <= part_delta <= 0.6253968253968254


In this attribute, too, moving pairs of empty values to negative values will result in a clearer distinction between pairs of uniques and duplicates, as will be seen in the graphical comparison of capter [Features Discussion and Dummy Classifier Baseline](./5_FeatureDiscussionDummyBaseline.ipynb).

In [52]:
df_feature_base = dpf.mark_missing(df_feature_base, 'part', factor)

### person

As a result of chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{person}$ has been split into three specific attributes. Attribute $\texttt{person}\_{100}$ and $\texttt{person}\_{700}$ hold strongly standardised string values. For comparing pure strings, a Levenshtein metric is recommended [[Chri2012](./A_References.ipynb#chri2012)]. Unfortunately, this metric shows a very long calculation time on the data of the capstone project. Comparing the similarity values of the Levenshtein metric with the similarity values of other metrics in appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb), similarity metric StrCmp95 has been decided to use.

In [53]:
attribute = 'person'

columns_metadata_dict['similarity_metrics'][attribute+'_100'] = tedi.StrCmp95()
columns_metadata_dict['similarity_metrics'][attribute+'_700'] = tedi.StrCmp95()
#tedi.Levenshtein()

pe_values = ['_100', '_700']

for pe in pe_values :
    print('Calculating person'+pe)
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+pe,
        columns_metadata_dict['similarity_metrics'][attribute+pe],
        columns_metadata_dict)

Calculating person_100


Calculating person_700


In [54]:
pe = '_100'

uniques[attribute+pe], uniques_len[attribute+pe] = dpf.determine_similarity_values(
    df_feature_base, attribute+pe)

person_100 values range [0.         0.31944444 0.32777778 0.33095238 0.34768519 0.3562963
 0.35714286 0.36414141 0.36527778 0.36723647 0.37777778 0.38603989
 0.38779956 0.38989899 0.39246032 0.39490741 0.40666667 0.41281046
 0.4130719  0.41596491 0.42324786 0.42388889 0.4240404  0.42486772
 0.42614379 0.42676768 0.42819444 0.42869281 0.42982456 0.42991453
 0.43326118 0.43415893 0.43508772 0.43660714 0.43794872 0.43804714
 0.439819   0.44040404 0.44079254 0.44126984 0.44222222 0.44282407
 0.44292929 0.44444444 0.44457516 0.44507937 0.44571078 0.44649123
 0.44656085 0.4465812  0.44662698 0.44722222 0.44782135 0.44786325
 0.44791667 0.4481352  0.44972222 0.45126263 0.45131313 0.45222222
 0.45291375 0.45314327 0.45416667 0.45424242 0.45555556 0.45564103
 0.45578704 0.45784314 0.45873016 0.45952381 0.46013072 0.46055556
 0.46119281 0.46127946 0.46336996 0.46428571 0.46452763 0.46452991
 0.46455026 0.46572872 0.46597222 0.46666667 0.46696833 0.46703704
 0.46813725 0.46978558 0.47007576 0.471

In [55]:
position = uniques_len[attribute+pe]

dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position-2],
    uniques[attribute+pe][uniques_len[attribute+pe]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position],
    uniques[attribute+pe][uniques_len[attribute+pe]-position+2], 10)

,duplicates,person_100_delta,person_100_x,person_100_y
8427,0,1.0,,
96724,0,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus
98329,0,1.0,,
51791,0,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus
2829,0,1.0,,
72990,0,1.0,,
85886,0,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus
54145,0,1.0,,
65592,0,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus
89750,0,1.0,,


0.8666666666666666 <= person_100_delta <= 1.0


,duplicates,person_100_delta,person_100_x,person_100_y
35148,0,0.0,,mozartwolfgang amadeus
32337,0,0.0,austenjane,
8373,0,0.0,,mozartwolfgang amadeus
15873,0,0.0,,austenjane
41100,0,0.0,,mozartwolfgang amadeus
1247,0,0.0,mozartwolfgang amadeus,
18262,0,0.0,,mozartwolfgang amadeus
26196,0,0.0,schneiderreto u.,
46329,0,0.0,,bührerwalter
62884,0,0.0,,rosoffmeg


0.0 <= person_100_delta <= 0.3277777777777777


For comparing person names, like in attribute $\texttt{person}\_{245c}$, a Jaro metric will be tested [[Chri2012](./A_References.ipynb#chri2012)].

In [56]:
pe = '_245c'

columns_metadata_dict['similarity_metrics'][attribute+pe] = tedi.Jaro()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute+pe,
    columns_metadata_dict['similarity_metrics'][attribute+pe],
    columns_metadata_dict)

In [57]:
uniques[attribute+pe], uniques_len[attribute+pe] = dpf.determine_similarity_values(
    df_feature_base, attribute+pe)

person_245c values range [0.         0.25901876 0.26388889 ... 0.98782267 0.99453552 1.        ]


In [58]:
position = uniques_len[attribute+pe]

dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position-2],
    uniques[attribute+pe][uniques_len[attribute+pe]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position],
    uniques[attribute+pe][uniques_len[attribute+pe]-position+2], 10)

,duplicates,person_245c_delta,person_245c_x,person_245c_y
65838,0,1.0,,
104573,1,1.0,,
98739,0,1.0,,
12855,0,1.0,sigrid kessler... [et al.] ; [éd.:] interkanto...,sigrid kessler... [et al.] ; [éd.:] interkanto...
104409,1,1.0,birgit spinath (hrsg.),birgit spinath (hrsg.)
83697,0,1.0,,
97199,0,1.0,,
68757,0,1.0,sigrid kessler... [et al.] ; [éd.:] interkanto...,sigrid kessler... [et al.] ; [éd.:] interkanto...
103281,1,1.0,andreas flury,andreas flury
20962,0,1.0,,


0.9945355191256832 <= person_245c_delta <= 1.0


,duplicates,person_245c_delta,person_245c_x,person_245c_y
5264,0,0.0,,sigrid kessler [u.a.]
34150,0,0.0,,"mortzfeld, peter ; raabe, paul"
81649,0,0.0,,[musik] von w. a. mozart ; klavierauszug nach ...
67342,0,0.0,,"mortzfeld, peter; raabe, paul"
72891,0,0.0,wolfgang amadeus mozart ; libretto: emanuel sc...,
61980,0,0.0,,sigrid kessler... [et al.] ; [hrsg.:] interkan...
95007,0,0.0,,g.h. dufour direxit ; h. müllhaupt sculpsit
37768,0,0.0,,g. h. dufour direxit ; h. müllhaupt sculpsit
1720,0,0.0,vorgelegt von ursula broder,
22257,0,0.0,,[walter bührer]


0.0 <= person_245c_delta <= 0.26388888888888884


The similarities of all three $\texttt{person}$ attributes are affected by empty values. These will be handled the same way as the attributes above.

In [59]:
pe_values = ['_100', '_245c', '_700']

for pe in pe_values :
    df_feature_base = dpf.mark_missing(df_feature_base, 'person'+pe, factor)

### pubinit

This attribute holds publisher strings that have a similar representation as attribute $\texttt{person}$. A Jaro metric will be used.

In [60]:
attribute = 'pubinit'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaro()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [61]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

pubinit values range [0.         0.25132275 0.25303644 ... 0.97849462 0.98404165 1.        ]


In [62]:
position = uniques_len[attribute]//3

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-1],
    uniques[attribute][uniques_len[attribute]-position], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-5],
    uniques[attribute][uniques_len[attribute]-1], 10)

,duplicates,pubinit_delta,pubinit_x,pubinit_y
37878,0,0.552222,k. alber,[eidg. landestopographie]
63120,0,0.552222,k. alber,[eidg. landestopographie]
33040,0,0.552222,k. alber,[eidg. landestopographie]
11939,0,0.552222,k. alber,[eidg. landestopographie]
15783,0,0.552222,k. alber,[eidg. landestopographie]
73633,0,0.552222,k. alber,[eidg. landestopographie]
71185,0,0.552222,k. alber,[eidg. landestopographie]
44821,0,0.552239,cambridge university press,"[drucker des jordanus, i.e. georg husner]"
25760,0,0.552222,k. alber,[eidg. landestopographie]
61983,0,0.552222,k. alber,[eidg. landestopographie]


0.5522222222222223 <= pubinit_delta <= 0.552238531435088


,duplicates,pubinit_delta,pubinit_x,pubinit_y
88659,0,1.0,,
65136,0,1.0,,
57054,0,1.0,,
62630,0,1.0,,
29380,0,1.0,,
94002,0,1.0,,
62720,0,1.0,,
65940,0,1.0,staatlicher lehrmittelverlag,staatlicher lehrmittelverlag
45588,0,1.0,,
5885,0,1.0,,


0.9711484593837535 <= pubinit_delta <= 1.0


The similarities of $\texttt{pubinit}$ are affected by empty values. These will be transformed to negative values.

In [63]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### scale

Comparing the similarity metrics of some sample value pairs of attribute $\texttt{scale}$ in appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb), a Jaccard metrics has been identified to express the best matching behaviour for purely numerical values stored in the attribute.

In [64]:
attribute = 'scale'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [65]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

scale values range [0.         0.04587156 0.05504587 0.57142857 1.        ]


In [66]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-3],
    uniques[attribute][uniques_len[attribute]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-4],
    uniques[attribute][uniques_len[attribute]-3], 10)

,duplicates,scale_delta,scale_x,scale_y
101353,0,0.0,,100000
22175,0,0.0,,50000
29929,0,0.0,,100000
64002,0,0.0,,100000
57365,0,0.0,,100000
16728,0,0.0,,50000
54112,0,0.0,,100000
76513,0,0.0,,50000
96093,0,0.0,,100000
88570,0,0.0,,100000


0.0 <= scale_delta <= 0.04587155963302747


,duplicates,scale_delta,scale_x,scale_y
26894,0,0.571429,50000,100000
99682,0,0.571429,50000,100000
36395,0,0.571429,50000,100000
49904,0,0.571429,100000,50000
40430,0,0.571429,50000,100000
17100,0,0.571429,50000,100000
76940,0,0.571429,100000,50000
36019,0,0.571429,50000,100000
58549,0,0.571429,50000,100000
60977,0,0.571429,50000,100000


0.05504587155963303 <= scale_delta <= 0.5714285714285714


,duplicates,scale_delta,scale_x,scale_y
28607,0,0.045872,Scala 1:50.000 ; proiezione cilindrica ad asse...,50000
57993,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000
54855,0,0.045872,Scala 1:50.000 ; proiezione cilindrica ad asse...,50000
104055,1,0.045872,Scala 1:50.000 ; proiezione cilindrica ad asse...,50000
104054,1,0.045872,Scala 1:50.000 ; proiezione cilindrica ad asse...,50000
26918,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000
104057,1,0.045872,Scala 1:50.000 ; proiezione cilindrica ad asse...,50000
104053,1,0.045872,50000,Scala 1:50.000 ; proiezione cilindrica ad asse...
57830,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000
79497,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000


0.04587155963302747 <= scale_delta <= 0.05504587155963303


Attribute $\texttt{scale}$ is filled for maps, only. Due to its sparse filling, the similarities of the attribute are affected strongly by empty values. These empty values will be marked with a special negative value.

In [67]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### ttlfull

Due to the discussion in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{ttlfull}$ has been split up into two new attributes $\texttt{ttlfull}\_\texttt{245}$ and $\texttt{ttlfull}\_\texttt{246}$ which will be compared by the same similarity metrics. A visual analysis of the values stored in the attribute, reveals a string of words, comparable to the strings in attribute $\texttt{person}\_\texttt{245c}$, above. The same similarity metric will be used for both title attributes, therefore.

In [68]:
attribute = 'ttlfull'

columns_metadata_dict['similarity_metrics'][attribute+'_245'] = tedi.Jaro()
columns_metadata_dict['similarity_metrics'][attribute+'_246'] = tedi.Jaro()

tf_values = ['_245', '_246']

for tf in tf_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+tf,
        columns_metadata_dict['similarity_metrics'][attribute+tf],
        columns_metadata_dict)

In [69]:
for tf in tf_values :
    uniques[attribute+tf], uniques_len[attribute+tf] = dpf.determine_similarity_values(
        df_feature_base, attribute+tf)

ttlfull_245 values range [0.         0.24579125 0.25714286 ... 0.99537037 0.99747475 1.        ]
ttlfull_246 values range [0.         0.35555556 0.36111111 0.36366612 0.36666667 0.40238095
 0.40555556 0.41190476 0.41296296 0.41313131 0.4168847  0.41923077
 0.41931217 0.42424242 0.42484886 0.43333333 0.43758865 0.43913043
 0.43998145 0.44242424 0.44507736 0.44539295 0.44722222 0.4479837
 0.44848485 0.44901497 0.45       0.45011338 0.45610717 0.45764895
 0.45833333 0.46094276 0.46205962 0.46527102 0.46729958 0.46885522
 0.46945938 0.47130647 0.47142857 0.47222222 0.472318   0.47267613
 0.47285513 0.47308219 0.47356964 0.47372609 0.47417027 0.47428613
 0.47431581 0.47446677 0.47462215 0.47491364 0.47508251 0.47515152
 0.47518634 0.47597067 0.47606838 0.47613256 0.47629884 0.47642654
 0.47829216 0.47883598 0.47923655 0.48002481 0.48055556 0.48072175
 0.48090015 0.48117181 0.48140097 0.48175055 0.48195938 0.48263045
 0.48306533 0.48413623 0.48439153 0.4856778  0.48578781 0.48604908
 0.48641

In [70]:
tf = '_245'
position = uniques_len[attribute+tf]

dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-position],
    uniques[attribute+tf][uniques_len[attribute+tf]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-3],
    uniques[attribute+tf][uniques_len[attribute+tf]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-4],
    uniques[attribute+tf][uniques_len[attribute+tf]-3], 10)

,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
61458,0,0.0,bildungsforschung und bildungspraxis,emma
73235,0,0.0,emma,how do i find the christ?
22869,0,0.0,emma,blick in die welt
79826,0,0.0,emma,blick in die welt
13256,0,0.0,arts,blick in die welt
30521,0,0.0,emma,blick in die welt
34308,0,0.0,emma,blick in die welt
75266,0,0.0,emma,trionfi
43643,0,0.0,emma,blick in die welt
62193,0,0.0,emma,blick in die welt


0.0 <= ttlfull_245_delta <= 0.24579124579124578


,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
103222,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
103241,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
103244,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
104542,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
103224,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
103229,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
103243,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
103262,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
103213,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
81489,0,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."


0.9953703703703703 <= ttlfull_245_delta <= 0.9974747474747474


,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
103215,1,0.995370,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
103262,1,0.995370,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
103266,1,0.995370,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
103223,1,0.995370,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
104534,1,0.995370,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
26832,0,0.995370,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
10212,0,0.994048,"bonne chance!, cours de langue française, deux...","bonne chance, cours de langue française, deuxi..."
103238,1,0.995370,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
103260,1,0.995370,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
104531,1,0.995370,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."


0.9940476190476191 <= ttlfull_245_delta <= 0.9953703703703703


Attribute $\texttt{ttlfull}\_\texttt{245}$ is filled for all data rows of Swissbib's raw data as can be seen in chapter [Data Analysis](./1_DataAnalysis.ipynb). For attribute $\texttt{ttlfull}\_\texttt{245}$, the filling is below $10\%$. The data pairs with missing values will be marked with a negative value as has been done for similar cases above.

In [71]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute+'_246', factor)

### volumes

This attribute is described in chapter [Data Analysis](./1_DataAnalysis.ipynb) to hold a kind of contents that resembles the contents of attribute $\texttt{part}$. The same similarity metrics will be used for attribute $\texttt{volumes}$ as for attribute $\texttt{part}$, therefore.

In [72]:
attribute = 'volumes'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.StrCmp95()
#tedi.Jaro()
#tedi.LCSSeq()
#tedi.MongeElkan()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [73]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

volumes values range [0.         0.25632184 0.26111111 ... 0.98484848 0.98765432 1.        ]


In [74]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-3],
    uniques[attribute][uniques_len[attribute]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-4],
    uniques[attribute][uniques_len[attribute]-3], 10)

,duplicates,volumes_delta,volumes_x,volumes_y
100427,0,0.0,64 s.,
102403,0,0.0,,[76] s.
17771,0,0.0,365 s.,partitur
3300,0,0.0,270 s.,
6476,0,0.0,239 p.,stimmen
62821,0,0.0,bd.,"135, 135 p."
73531,0,0.0,online-ressource,
78384,0,0.0,v.,327 s.
43897,0,0.0,1 (klavier-auszug),
52846,0,0.0,,1 coffret de 2 cd (ddd)


0.0 <= volumes_delta <= 0.2563218390804598


,duplicates,volumes_delta,volumes_x,volumes_y
103465,1,0.987654,2 dvd-video (ca. 109 min.),2 dvd-videos (ca. 109 min.)
103453,1,0.987654,2 dvd-videos (ca. 109 min.),2 dvd-video (ca. 109 min.)
103477,1,0.987654,2 dvd-videos (ca. 109 min.),2 dvd-video (ca. 109 min.)
79094,0,0.984848,1 dvd-video (169 min.),1 dvd-video (16 min.)
103468,1,0.987654,2 dvd-video (ca. 109 min.),2 dvd-videos (ca. 109 min.)
103486,1,0.987654,2 dvd-video (ca. 109 min.),2 dvd-videos (ca. 109 min.)
103456,1,0.987654,2 dvd-videos (ca. 109 min.),2 dvd-video (ca. 109 min.)
103489,1,0.987654,2 dvd-video (ca. 109 min.),2 dvd-videos (ca. 109 min.)
103474,1,0.987654,2 dvd-videos (ca. 109 min.),2 dvd-video (ca. 109 min.)


0.9848484848484849 <= volumes_delta <= 0.9876543209876543


,duplicates,volumes_delta,volumes_x,volumes_y
51350,0,0.975000,online-ressource,online ressource
79094,0,0.984848,1 dvd-video (169 min.),1 dvd-video (16 min.)


0.975 <= volumes_delta <= 0.9848484848484849


Attribute $\texttt{volumes}$ holds rows with missing data. The data pairs with missing values will be marked with a special negative value.

In [75]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

## DataFrame with Attributes and Similarity Features

The metric for each attribute of the feature DataFrame has been decided and the similarity features have been calculated. In this last step, the columns of the DataFrame are reordered in order to place the $\_\texttt{delta}$ columns close to their input origins $\_\texttt{x}$ and $\_\texttt{y}$ and some sample records are shown.

In [76]:
# Take _x, _y, and _delta columns together
fb_col_list = df_feature_base.columns.tolist()
fb_col_list.sort()
# Move target column to first place
fb_col_list.insert(0, fb_col_list.pop(fb_col_list.index('duplicates')))
# Reorder DataFrame columns
df_attribute_with_sim_feature = pd.DataFrame(df_feature_base, columns=fb_col_list)

# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(df_attribute_with_sim_feature.columns)

class_label = ['uniques', 'duplicate']

for i in class_label:
    display(df_attribute_with_sim_feature[df_attribute_with_sim_feature.duplicates==class_label.index(i)].sample(n=10))
    print(i)

,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y,coordinate_N_delta,coordinate_N_x,coordinate_N_y,corporate_full_delta,corporate_full_x,corporate_full_y,doi_delta,doi_x,doi_y,edition_delta,edition_x,edition_y,exactDate_delta,exactDate_x,exactDate_y,format_postfix_delta,format_postfix_x,format_postfix_y,format_prefix_delta,format_prefix_x,format_prefix_y,isbn_delta,isbn_x,isbn_y,ismn_delta,ismn_x,ismn_y,musicid_delta,musicid_x,musicid_y,part_delta,part_x,part_y,person_100_delta,person_100_x,person_100_y,person_245c_delta,person_245c_x,person_245c_y,person_700_delta,person_700_x,person_700_y,pubinit_delta,pubinit_x,pubinit_y,scale_delta,scale_x,scale_y,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y,ttlfull_246_delta,ttlfull_246_x,ttlfull_246_y,volumes_delta,volumes_x,volumes_y
38949,0,-0.5,e0080855,,-0.5,n0460826,,-0.500000,eidgenössische landestopographie,,-1.0,,,0.0,1,4,0.250,1963aaaa,2005uuuu,0.111111,010300,020053,0.0,mp,bk,0.0,[],"[978-3-13-127284-3, 3-13-127284-8]",-1.0,,,-1.0,,,-0.500000,"bl. 285, 285,1963",,-0.500000,,schusterhans-peter,-0.500000,,"hans-peter schuster, hans-joachim trappe",-0.500000,,trappehans-joachim,0.442656,servizio topografico federale,g. thieme,-0.5,50000,,0.476874,domodossola,ekg-kurs für isabel,-1.0,,,0.463393,1 carta,online-ressource
68478,0,-1.0,,,-1.0,,,-1.000000,,,-1.0,,,-0.5,,3,0.500,19739999,1992uuuu,0.111111,030500,020000,0.0,cr,bk,0.0,[0335-1793],[],-1.0,,,-1.0,,,-0.500000,,1,-1.000000,,,-0.500000,,sigrid kessler... [et al.] ; [éd.:] interkanto...,-1.000000,,,0.493651,libération,staatlicher lehrmittelverlag,-1.0,,,0.516435,libération,"bonne chance!, cours de langue française, troi...",-1.0,,,-0.500000,,237 p.
35530,0,-1.0,,,-1.0,,,0.016667,schweiz,"interkantonale lehrmittelzentrale (rapperswil,...",-1.0,,,-0.5,6,,0.500,20159999,19989999,1.000000,020000,020000,1.0,bk,bk,0.0,[978-3-7190-3171-8 (Bd. 1)],[],-1.0,,,-1.0,,,-1.000000,,,-1.000000,,,0.581391,hrsg.: heinrich honsell ... [et al.],sigrid kessler... [et al.] ; [hrsg.:] interkan...,0.514687,"honsellheinrich, ammanncaterina",kesslersigrid,0.547828,helbing lichtenhahn,staatlicher lehrmittelverlag,-1.0,,,0.482776,obligationenrecht,"bonne chance!, cours de langue française, 3",-0.5,or,,1.000000,v.,v.
70650,0,-1.0,,,-1.0,,,-1.000000,,,-1.0,,,-0.5,,2,0.375,1959aaaa,2009uuuu,1.000000,020000,020000,1.0,bk,bk,0.0,[],[978-3-596-90041-1],-1.0,,,-1.0,,,0.000000,"no 27, vol. 597, 27, 597",90041,0.514722,voltaire,austenjane,0.520609,voltaire ; préf. dominique aury,jane austen ; mit einem nachw. von rudolf sühnel,-1.000000,,,0.432479,guilde du livre,fischer-taschenbuch-verlag,-1.0,,,0.490997,traité sur la tolérance,"emma, roman",-1.0,,,0.555556,214 p.,509 s.
96156,0,-1.0,,,-1.0,,,-0.500000,,arts florissants,-1.0,,,-1.0,,,0.500,1979aaaa,1996uuuu,0.428571,020000,040000,0.0,bk,mu,1.0,[],[],-1.0,,,-0.5,,0630,-1.000000,,,0.581818,austenjane,mozartwolfgang amadeus,0.505051,jane austen,mozart,-0.500000,,"christiewilliam, dessaynatalie, mannionrosa, b...",-0.500000,,erato,-1.0,,,0.528986,emma,"die zauberflöte, kv 620",-1.0,,,0.485842,549 s.,2 disques compacts en 1 coffret
95103,0,-1.0,,,-1.0,,,-0.500000,,"staatsoper (wien)chor, wiener philharmoniker",-1.0,,,-1.0,,,0.500,2005aaaa,uuuuuuuu,0.111111,010100,040000,1.0,mu,mu,1.0,[],[],-0.5,m006204687,,0.0,601,38,-1.000000,,,1.000000,mozartwolfgang amadeus,mozartwolfgang amadeus,0.416667,w. a. mozart,mozart,0.547378,mozartwolfgang amadeus,"böhmekurt, berrywalter, della casalisa, donchk...",0.529040,bärenreiter,penzance records,-1.0,,,0.884058,"die zauberflöte, kv 620",die zauberflöte,-1.0,,,0.500585,"1 partition de travail (xxvii, 379 p.)",3 disques 33 tours
38294,0,-1.0,,,-1.0,,,-1.000000,,,-0.5,10.1093/cid/ciu795,,-1.0,,,0.750,2015aaaa,2015uuuu,0.111111,010053,020000,1.0,bk,bk,0.0,[],[978-0-7294-1156-1],-1.0,,,-1.0,,,0.423977,"60/3(2015), 432-437",13b,-0.500000,,voltaire,0.566972,"[v. rozot, a. patrizia, s. vigano, j. mazza-st...",voltaire ; [sous la dir. de diego venturino],0.573690,"rozotv., patrizi

uniques


,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y,coordinate_N_delta,coordinate_N_x,coordinate_N_y,corporate_full_delta,corporate_full_x,corporate_full_y,doi_delta,doi_x,doi_y,edition_delta,edition_x,edition_y,exactDate_delta,exactDate_x,exactDate_y,format_postfix_delta,format_postfix_x,format_postfix_y,format_prefix_delta,format_prefix_x,format_prefix_y,isbn_delta,isbn_x,isbn_y,ismn_delta,ismn_x,ismn_y,musicid_delta,musicid_x,musicid_y,part_delta,part_x,part_y,person_100_delta,person_100_x,person_100_y,person_245c_delta,person_245c_x,person_245c_y,person_700_delta,person_700_x,person_700_y,pubinit_delta,pubinit_x,pubinit_y,scale_delta,scale_x,scale_y,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y,ttlfull_246_delta,ttlfull_246_x,ttlfull_246_y,volumes_delta,volumes_x,volumes_y
103721,1,-1.0,,,-1.0,,,-1.00,,,-1.0,,,-1.0,,,0.75,1970aaaa,1970uuuu,1.0,010200,010200,1.0,mu,mu,1.0,[],[],-1.0,,,1.0,4553,4553,-1.0,,,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus,0.791326,wolfgang amadeus mozart ; text von emanuel sch...,text von emanuel schikaneder ; [musik von] wol...,0.706061,"grubergernot, orelalfred, moehnheinz, schikane...","moehnheinz, schikanederemanuel, grubergernot",1.000000,bärenreiter,bärenreiter,-1.0,,,1.000000,"die zauberflöte, eine deutsche oper in zwei au...","die zauberflöte, eine deutsche oper in zwei au...",-1.0,,,0.507778,1 réduction (221 p.),1 klavierauszug
103286,1,-1.0,,,-1.0,,,-1.00,,,-1.0,,,-1.0,,,0.75,1999aaaa,1999uuuu,1.0,020000,020000,1.0,bk,bk,1.0,[3-495-47879-5],[3-495-47879-5],-1.0,,,-1.0,,,1.0,"bd. 57, 57","bd. 57, 57",1.0,fluryandreas,fluryandreas,1.000000,andreas flury,andreas flury,-1.000000,,,1.000000,k. alber,k. alber,-1.0,,,1.000000,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p...",-1.0,,,1.000000,316 s.,316 s.
104424,1,-1.0,,,-1.0,,,-1.00,,,-1.0,,,-1.0,,,0.75,2014aaaa,2014uuuu,1.0,020000,020000,1.0,bk,bk,1.0,"[978-3-642-41697-2, 3-642-41697-7]","[978-3-642-41697-2, 3-642-41697-7]",-1.0,,,-1.0,,,-1.0,,,-1.0,,,1.000000,birgit spinath (hrsg.),birgit spinath (hrsg.),1.000000,spinathbirgit,spinathbirgit,-1.000000,,,-1.0,,,1.000000,"empirische bildungsforschung, aktuelle themen ...","empirische bildungsforschung, aktuelle themen ...",-1.0,,,1.000000,"xi, 158 s.","xi, 158 s."
103564,1,-1.0,,,-1.0,,,-1.00,,,-1.0,,,-1.0,,,0.75,2005aaaa,2005uuuu,1.0,020000,020000,1.0,bk,bk,1.0,[0-521-82437-0],"[978-0-521-82437-8, 0-521-82437-0]",-1.0,,,-1.0,,,-1.0,,,1.0,austenjane,austenjane,1.000000,jane austen ; ed. by richard cronin ... [et al.],jane austen ; ed. by richard cronin ... [et al.],-0.500000,,croninrichard,-0.500000,,cambridge university press,-1.0,,,1.000000,emma,emma,-1.0,,,0.666667,599 s.,600 s.
104489,1,-1.0,,,-1.0,,,-1.00,,,-1.0,,,-1.0,,,0.75,2015aaaa,2015uuuu,1.0,020000,020000,1.0,bk,bk,1.0,[978-0-7294-1156-1],[978-0-7294-1156-1],-1.0,,,-1.0,,,1.0,13 b,13 b,1.0,voltaire,voltaire,0.981818,sous la dir. de diego venturino = [les oeuvres...,sous la dir. de diego venturino = [les oeuvres...,1.000000,venturinodiego,venturinodiego,1.000000,voltaire foundation,voltaire foundation,-1.0,,,1.000000,"the complete works of voltaire, siècle de loui...","the complete works of voltaire, siècle de loui...",-1.0,,,1.000000,453 p.,453 p.
103365,1,-1.0,,,-1.0,,,1.00,schweizerische gesellschaft für bildungsforschung,schweizerische gesellschaft für bildungsforschung,-1.0,,,-1.0,,,1.00,19791999,19791999,1.0,030600,030600,1.0,cr,cr,1.0,[0252-9955],[0252-9955],-1.0,,,-1.0,,,-1.0,,,-1.0,,,0.673360,hrsg.: schweizerische gesellschaft für bildung...,hrsg. schweizerische gesellschaft für bildungs...,-1.000000,,,-0.500000,,universitätsverlag,-1.0,,,0.798535,bildungsforschung und bildungspraxis,"bildungsforschung und bildungspraxis, schweize...",-0.5,educazione e ricerca,,-1.000000,,
104237,1,-1.0,,,-1.0,,,-1.00,,,-1.0,,,-1.0,,,0.75,2001aaaa,2001uuuu,1.0,020000,020000,1.0,bk,bk,1.0,[0-375-75742-2],[0-375-75742-2],-1.0,,,-1.0,,,-1.0,,,1.0,austenjane,austenjane,0.725806,j

duplicate


## Summary

This chapter covers the central area of feature construction. The features of the feature matrix have been generated for each attribute of Swissbib's raw data, deciding on its similarity metric. With these metric values, the feature base DataFrame has been extended and a new DataFrame with the attribute values of the pairs together with their calculated similarity value have been generated. The similarity values will be the final features for training and performance testing of the models, compare [[JudACaps](./A_References.ipynb#judacaps)].

In [77]:
columns_metadata_dict['similarity_metrics']

{'coordinate_E': LCSStr({'qval': 1, 'external': True}),
 'coordinate_N': LCSStr({'qval': 1, 'external': True}),
 'corporate_full': LCSStr({'qval': 1, 'external': True}),
 'doi': Identity({'qval': 1, 'external': True}),
 'edition': Jaccard({'qval': 1, 'as_set': False, 'external': True}),
 'exactDate': Hamming({'qval': 1, 'test_func': <function Base._ident at 0x11a300510>, 'truncate': False, 'external': True}),
 'format_prefix': Identity({'qval': 1, 'external': True}),
 'format_postfix': Jaccard({'qval': 2, 'as_set': False, 'external': True}),
 'isbn': Identity({'qval': 1, 'external': True}),
 'ismn': Identity({'qval': 1, 'external': True}),
 'musicid': LCSStr({'qval': 1, 'external': True}),
 'part': StrCmp95({'long_strings': False, 'external': True}),
 'person_100': StrCmp95({'long_strings': False, 'external': True}),
 'person_700': StrCmp95({'long_strings': False, 'external': True}),
 'person_245c': Jaro({'qval': 1, 'long_tolerance': False, 'winklerize': False, 'external': True}),
 'pu

The similarity metric decided for each attribute has been added as an additional piece of information to the columns metadata dictionary. The following table gives this summary in a structured form and lists the metric used for each attribute. Attributes with the same font color indicate similar types of values (description column) for better orientation.

| attribute     | subtype | description | similarity metric |
| ------------- |:--------|:------------|:------------------|
|<font color='red'>[coordinate](#coordinate)</font>|<font color='red'>\_E</font>|<font color='red'>Code(9)</font>|<font color='red'>LCSStr</font>|
|               |<font color='red'>\_N</font>|<font color='red'>Code(9)</font>|<font color='red'>LCSStr</font>|
|<font color='blue'>[corporate](#corporate)</font>|<font color='blue'>\_full</font>|<font color='blue'>Name</font>|<font color='blue'>LCSStr</font>|
|<font color='green'>[doi](#doi)</font>|         |<font color='green'>Identifier</font>|<font color='green'>Identity</font>|
|<font color='orange'>[edition](#edition)</font>|         |<font color='orange'>Number</font>|<font color='orange'>Jaccard</font>|
|<font color='black'>[exactDate](#exactDate)</font>|         |<font color='black'>Date</font>|<font color='black'>Hamming</font>|
|<font color='red'>[format](#format)</font>|<font color='red'>\_prefix</font>|<font color='red'>Code(2)</font>|<font color='red'>Identity</font>|
|               |<font color='red'>\_postfix</font>|<font color='red'>Code(6)</font>|<font color='red'>Jaccard (qval=2)</font>|
|<font color='green'>[isbn](#isbn)</font>|         |<font color='green'>Identifier</font>|<font color='green'>Identity</font>|
|<font color='green'>[ismn](#ismn)</font>|         |<font color='green'>Identifier</font>|<font color='green'>Identity</font>|
|<font color='green'>[musicid](#musicid)</font>|         |<font color='green'>Identifier</font>|<font color='green'>LCSStr</font>|
|<font color='orange'>[part](#part)</font>|         |<font color='orange'>Number</font>|<font color='orange'>StrCmp95</font>|
|<font color='blue'>[person](#person)</font>|<font color='blue'>\_100</font>|<font color='blue'>Name</font>|<font color='blue'>StrCmp95</font>|
|               |<font color='blue'>\_700</font>|<font color='blue'>Name</font>|<font color='blue'>StrCmp95</font>|
|               |<font color='blue'>\_245c</font>|<font color='blue'>Name</font>|<font color='blue'>Jaro</font>|
|<font color='blue'>[pubinit](#pubinit)</font>|         |<font color='blue'>Name</font>|<font color='blue'>Jaro</font>|
|<font color='orange'>[scale](#scale)</font>|         |<font color='orange'>Number</font>|<font color='orange'>Jaccard</font>|
|<font color='blue'>[ttlfull](#ttlfull)</font>|<font color='blue'>\_245</font>|<font color='blue'>String</font>|<font color='blue'>Jaro</font>|
|               |<font color='blue'>\_246</font>|<font color='blue'>String</font>|<font color='blue'>Jaro</font>|
|<font color='orange'>[volumes](#volumes)</font>|         |<font color='orange'>Number</font>|<font color='orange'>StrCmp95</font>|

### Full Feature Matrix with Target Vector Handover

To hand over the resulting DataFrame of this chapter, the DataFrame is saved into a pickle file that will be read in the next chapters [Features Discussion and Dummy Classifier Baseline](./5_FeatureDiscussionDummyBaseline.ipynb) as input.

In [78]:
# Store into compressed intermediary file
with bz2.BZ2File(os.path.join(path_goldstandard,
                       'labelled_feature_matrix_full.pkl'), 'w') as df_output_file:
    pk.dump(df_attribute_with_sim_feature, df_output_file)

The full metadata dictionary is to be persisted for handover to subsequent chapters.

In [79]:
# The target is still needed for the feature matrix
columns_metadata_dict['features'].append('duplicates')

for k in columns_metadata_dict.keys():
    print(k, '\n', columns_metadata_dict[k], '\n')

data_analysis_columns 
 ['coordinate_E', 'coordinate_N', 'corporate_full', 'doi', 'edition', 'exactDate', 'format_prefix', 'format_postfix', 'isbn', 'ismn', 'musicid', 'part', 'person_100', 'person_700', 'person_245c', 'pubinit', 'scale', 'ttlfull_245', 'ttlfull_246', 'volumes'] 

columns_to_use 
 ['duplicates', 'coordinate_E_x', 'coordinate_E_y', 'coordinate_N_x', 'coordinate_N_y', 'corporate_full_x', 'corporate_full_y', 'doi_x', 'doi_y', 'edition_x', 'edition_y', 'exactDate_x', 'exactDate_y', 'format_prefix_x', 'format_prefix_y', 'format_postfix_x', 'format_postfix_y', 'isbn_x', 'isbn_y', 'ismn_x', 'ismn_y', 'musicid_x', 'musicid_y', 'part_x', 'part_y', 'person_100_x', 'person_100_y', 'person_700_x', 'person_700_y', 'person_245c_x', 'person_245c_y', 'pubinit_x', 'pubinit_y', 'scale_x', 'scale_y', 'ttlfull_245_x', 'ttlfull_245_y', 'ttlfull_246_x', 'ttlfull_246_y', 'volumes_x', 'volumes_y'] 

similarity_metrics 
 {'coordinate_E': LCSStr({'qval': 1, 'external': True}), 'coordinate_N': L

In [80]:
# Binary intermediary metadata file
with open(os.path.join(path_goldstandard,
                       'columns_metadata.pkl'), 'wb') as dict_output_file:
    pk.dump(columns_metadata_dict, dict_output_file)